# 06. 이미지 회전과 원근 변환

In [ ]:
import cv2 as cv
import numpy as np

DOG_PATH = "../images/dog.jpg"
CARD_PATH = "../images/card.jpg"

## 6-1. 이미지 회전(rotate)
- `cv2.rotate(img, rotateCode)`
- `rotateCode`
    - `cv2.ROTATE_90_CLOCKWISE` : 시계방향 90도 회전
    - `cv2.ROTATE_180` : 180도 회전
    - `cv2.ROTATE_90_COUNTERCLOCKWISE` : 반시계방향 90도 회전

In [ ]:
img = cv.imread(DOG_PATH)

dst_90 = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
dst_180 = cv.rotate(img, cv.ROTATE_180)
dst_90_counter = cv.rotate(img, cv.ROTATE_90_COUNTERCLOCKWISE)

cv.imshow("original", img)
cv.imshow("90", dst_90)
cv.imshow("180", dst_180)
cv.imshow("90 counter", dst_90_counter)

cv.waitKey(0)
cv.destroyAllWindows()

## 6-2. 이미지 회전(아핀 변환)
- 행렬 연산을 사용하여 정의
- cv2.wrapAffine
- cv.getRotationMatrix2D

In [ ]:
# 아핀 변환(math 모듈로 직접 구현)
import math

img = cv.imread(DOG_PATH)

rad = 45 * math.pi / 180
affine = np.array([[math.cos(rad), -math.sin(rad), 0], 
                [math.sin(rad), math.cos(rad), 0]])

dst = cv.warpAffine(img, affine, (img.shape[1], img.shape[0]))

cv.imshow("img", img)
cv.imshow("rotate", dst)

cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# 아핀 변환(getRotationMatrx2D 이용)
img = cv.imread(DOG_PATH)

center = (int(img.shape[1]/2), int(img.shape[0]/2))
scale = (img.shape[1], img.shape[0])
affine = cv.getRotationMatrix2D(center, 90, 1)
dst = cv.warpAffine(img, affine, scale)

cv.imshow("img", img)
cv.imshow("rotate", dst)

cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# 트랙바와 함께 사용
img = cv.imread(DOG_PATH)

name = "Rotation"
cv.namedWindow(name)
trackbar_name = "angle"
cv.createTrackbar(trackbar_name, name, 0, 360, lambda x:x)


while True:
    angle = cv.getTrackbarPos(trackbar_name, name)
    center = (int(img.shape[1]/2), int(img.shape[0]/2))
    scale = (img.shape[1], img.shape[0])
    affine = cv.getRotationMatrix2D(center, angle, 1)
    dst = cv.warpAffine(img, affine, scale)

    cv.imshow(name, dst)
    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()

## 6-3. 원근 변환
- 이미지에서 특정 부분의 기하학적 왜곡을 수정하거나 시점을 변경
- `cv2.getPerspectiveTransform(src, dst)`
    - src : 원래 위치
    - dst : 변환하고 싶은 위치
- `cv2.warpPerspective(img, mat, (width, height))`

In [ ]:
img = cv.imread(CARD_PATH)

width, height = 600, 350

# 원근을 변형할 지점 선택 : 시계 방향
src = np.array([[56,260], [981,128], [1213,560], [194, 735]], dtype=np.float32)

# 결과물을 출력할 형태 : 시계방향
dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

# 원근을 변형하기 위한 Matrix 생성
mat = cv.getPerspectiveTransform(src, dst)

# img를 mat에 의해 변환
result = cv.warpPerspective(img, mat, (width, height))

cv.imshow("img", img)
cv.imshow("warped", result)

cv.waitKey(0)
cv.destroyAllWindows()


In [6]:
# 실습1. 이미지 원근 변환하기
PRACTICE_CARD_PATH = "../images/practice_card.jpg"

img = cv.imread(PRACTICE_CARD_PATH)
resized =cv.resize(img, None, fx=0.15, fy=0.15)

width, height = 400, 250

src = np.array([[846,3027], [1994,1987], [2729,2682], [1563, 3802]], dtype=np.float32)

dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

mat = cv.getPerspectiveTransform(src, dst)

result = cv.warpPerspective(img, mat, (width, height))

cv.imshow("resized", resized)
cv.imshow("warped", result)

cv.waitKey(0)
cv.destroyAllWindows()

## 6-4. 마우스 이벤트
- `cv2.setMouseCallback(windowName, onMouse)` 
    - `onMouse(event)` : 마우스 이벤트를 처리하는 콜백함수, 첫 번째 인자로 이벤트를 받아줌

- 마우스 이벤트 플래그 : cv::MouseEventFlags
    - 마우스 이벤트는 "어떤 동작이 발생했는가"를 나타냄
    - 플래그는 "그 동작이 일어날 때 어떤 버튼이나 키가 눌려 있었는가"를 나타냄

In [5]:
img = cv.imread(DOG_PATH)
name = "Mouse Event"
cv.namedWindow(name)

def on_mouse(event, x, y, flags, _):
    # print(event, x, y, flags)
    if event == cv.EVENT_LBUTTONDOWN:
        copied = img.copy() # 원본 그대로 복사본에만 찍힘
        cv.circle(copied, (x, y), 20, (255,255,0), cv.FILLED)
        cv.imshow(name, copied)

cv.setMouseCallback(name, on_mouse)
cv.imshow(name, img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# 실습2. 반자동 문서 스캐너 만들기
# [구현 내용]
# 1. 이미지를 시계방향으로 클릭하면
# 2. 이미지에 점이 찍히고
# 3. 점과 점 사이에 라인이 연결됨
    # 가능하면 라인이 마우스를 따라가도록 구현
# 4. 4개의 점을 찍으면 해당 영역이 펼쳐짐
# [구현 우선순위]
# 1. 네 개의 지점을 선택하면 원근 변환이 실행
# 2. 점과 점 사이를 라인으로 연결
# 3. 라인이 마우스를 따라오도록 구현

img = cv.imread(CARD_PATH)

name = "Scanner" # 창 이름
point_list = [] # 클릭한 좌표 저장할 리스트

# 원근 변환을 실행, point_list에 저장된 4갱의 점을 기준으로 선택한 영역을 사각형으로 펴줌
def show_result():
    # 첫 번째 점과 두 번째 점 사이의 거리 
    width = int(np.linalg.norm(np.array(point_list[0]) - np.array(point_list[1])))
    # 첫 번째 점과 네 번째 점 사이의 거리
    height = int(np.linalg.norm(np.array(point_list[0]) - np.array(point_list[3])))

    # 원본에서의 4개 좌표
    src = np.array(point_list, dtype=np.float32)
    # 결과 이미지를 만들 기준이 될 좌표
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)
    # 원근 변환 행렬 계산
    mat = cv.getPerspectiveTransform(src, dst)
    # 원근 변환 적용
    result = cv.warpPerspective(img, mat, (width, height))
    # 변환된 결과를 새로운 창에 출력
    cv.imshow("Cropped", result)

# 드래그 상태를 확인하기 위한 변수
drawing = False
# 마우스 이벤트 콜백 함수, 마우스로 클릭한 좌표를 point_list에 저장하고 클릭 순서대로 선으로 연결해가면 표시
def on_mouse(event, x, y, flags, _):
    global drawing
    coppied = img.copy()

    # 마우스 왼쪽 버튼 클릭시
    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        point = (x,y)
        point_list.append(point) # 클릭한 좌표 저장

    # 클릭 후 동작
    if drawing:
        prev_point = None # 이전 점 저장 변수
        for p in point_list: # 저장된 모든 좌표 순회
            cv.circle(coppied, p, 10,(0,255,255), cv.FILLED) # 클릭 된 점 표시

            # 이전 점이 존재하면 선 연결
            if prev_point:
                cv.line(coppied, prev_point, p, (0,255,255), 5)

            prev_point = p

        next_point = (x,y) # 실시간 연결선
        # 점 4개가 모두 선택되면 마지막 점과 첫 점 연결
        if len(point_list) == 4:
            next_point = point_list[0]
            show_result()
        # 마지막 점과 다음 점 연결선 그리기
        cv.line(coppied, point_list[-1], next_point, (0, 255, 255), 5)
    cv.imshow(name, coppied)

cv.namedWindow(name)
cv.setMouseCallback(name, on_mouse)
cv.imshow(name, img)
cv.waitKey(0)
cv.destroyAllWindows()